<a href="https://colab.research.google.com/github/aleksandr329/SCAN_project/blob/Anna_branch/SCAN_%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D1%82%D0%B8%D0%B5_%D1%84%D0%B0%D0%B9%D0%BB%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
from pprint import pprint
import re
from collections import Counter

# Открытие датасета.

In [2]:
# для открытия и чтения файла с данным c Google.Drive через Google.Colab

from google.colab import drive
drive.mount('/content/drive')
path_DF = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/Queries.txt'

Mounted at /content/drive


In [2]:
path_DF = 'DataSET SCAN/Queries.txt'

In [3]:
# В файле Queries.txt приведено множество поисковых запросов клиентов продукта СКАН
# (формат: UserId Query - разделитель табуляция).

data = pd.read_csv(path_DF, sep='\t', header=None, names=['UserId', 'Query'])
data.head()

,UserId,Query
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC..."
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(..."
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3..."
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28371 entries, 0 to 28370
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UserId  28371 non-null  object
 1   Query   28371 non-null  object
dtypes: object(2)
memory usage: 443.4+ KB


In [5]:
data.describe(include=object)

,UserId,Query
count,28371,28371
unique,1085,27593
top,00000000-0000-0000-0000-000000000000,"((Exact(Text(""9ff8d3d3-7624-490e-acd8-bfb2e3cf..."
freq,2219,7


In [6]:
print(f'Найдено дубликатов: {data.duplicated().sum()}')
print(f'Найдено пропусков в данных:\n {data.isna().sum()}')

Найдено дубликатов: 0
Найдено пропусков в данных:
 UserId    0
Query     0
dtype: int64


# Открытие json.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_JSON = '/content/drive/MyDrive/Data_Science/Хакатон_SCAN/ThemesWordClouds.json'

In [7]:
path_JSON = 'DataSET SCAN/ThemesWordClouds.json'

In [11]:

with open(path_JSON, encoding="utf8") as f:                     # Открываем файл и связываем его с объектом "f"
  words_clouds_data = json.load(f)                           # Загружаем содержимое открытого файла в переменную words_clouds_data

# Открываем как датафрейм
#wc_data = pd.DataFrame(words_clouds_data)

pprint(words_clouds_data[15]['WordCloud'])   #Ключей 572 (от 0 до 571)

['займы',
 'кредиты',
 'кредит',
 'заем',
 'ссуда',
 'кредитный',
 'кредитование',
 'программа',
 'линия',
 'предложение',
 'пакет',
 'портфель',
 'договор',
 'соглашение',
 'фабрика',
 'сервис',
 'сумма',
 'объем',
 'получение',
 'кредита',
 'получить',
 'привлечь',
 'банки',
 'банк']


In [12]:
wc_data = pd.DataFrame(words_clouds_data)
wc_data.head()

,Theme,WordCloud
0,"Письма, указы, заявления от госорганов","[письмо, указ, поправка, заявление, сообщение,..."
1,Истец,"[истец, после, основании, иск, иск, суд, требо..."
2,Ответчик,"[против, ответчик, иск, действие, поведение, с..."
3,Преступник,"[преступник, уличить, уличать, разыскивать, об..."
4,Потерпевшая сторона,"[потерпевшая, сторона, покушение, нападение, о..."


In [13]:
print(f'Количество уникальных тематик в облаках слов: {wc_data.Theme.nunique()}')

Количество уникальных тематик в облаках слов: 572


# Прочее.

In [14]:
# Фразовые запросы

phrase_queries = ['OrderedText', 'Text', 'OrderedWildcard', 'Wildcard',
                  'OrderedTextAnyDistance', 'InSameSentence', 'InSameParagraph',
                  'Exact']

# Запросы объектов и размерных величин
mesure_queries = ['Id', 'Metadata', 'Name', 'Org', 'Location', 'Trademark',
                  'IndustrialSector', 'IndustrialSectorPositiveTonality',
                  'IndustrialSectorNegativeTonality', 'NewsCluster', 'Date',
                  'Dim', 'AuthorName']

# Запросы фактов, рубрик и прямой речи
fact_queries = ['Speech', 'Event', 'Subject', 'EventType']

# Запросы источников
source_queries = ['Source', 'SourceLevel', 'SourceCategory', 'SourceRegion', 'SourceTopic']

# Остальное
others_queries = ['Language', 'DocumentType']

# Извлечение текста запроса (в работе).

In [15]:
df = data.copy()

In [16]:
df.UserId.nunique()

1085

In [48]:
def extract_strings(sql_query):
    pattern = r'Text\("(.*?)"\)'  # Регулярное выражение для поиска строк в формате Text("слово")
    matches = re.findall(pattern, sql_query)
    return matches

In [49]:
def extract_not_strings(sql_query):
    pattern = r'NOT\(Text\("(.*?)"\)'  # Регулярное выражение для поиска строк в формате Text("слово")
    matches = re.findall(pattern, sql_query)
    return matches

In [21]:
data[0:1].Query

0    ((Text("речные круизы")) AND (Metadata(SourceC...
Name: Query, dtype: object

In [27]:
# # Пример использования
# sql_query = '''(((Text("питерский экономический форум")) OR (Text("spief-2021")) OR 
# (Text(1,"экономический форум Петербург")) OR (Org(Id(1472700))) OR 
# (Text("Петербургского международного экономического форума")) OR (Text("SPIEF")) OR 
# (Text("русский давос"))) AND (Metadata(SourceCategory({13,14,3,4,2,7,1A,B,D,E})) AND NOT(DocumentType({0,1,3,5}))))
# '''
# strings = extract_strings(sql_query)
# print(type (strings))
# print(strings)

<class 'list'>
['питерский экономический форум', 'spief-2021', 'Петербургского международного экономического форума', 'SPIEF', 'русский давос']


In [50]:
df['QueryText'] = df['Query'].apply(lambda query: extract_strings(query))

In [57]:
df['QueryExcludeText'] = df['Query'].apply(lambda query: extract_not_strings(query))

In [58]:
df.head(10)

,UserId,Query,QueryText,QueryExcludeText
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC...",[речные круизы],[]
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(...",[Поезд],[]
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""...","[Яндекс.Директ, на наш Telegram, Выделите фраг...",[]
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3...",[отели],[]
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B...",[российских],[]
5,00000000-0000-0000-0000-000000000000,"((Text(""Москва"")) AND (Metadata(SourceCategory...",[Москва],[]
6,00000000-0000-0000-0000-000000000000,"((Text(""Москва"")) AND (Metadata(SourceCategory...",[Москва],[]
7,00000000-0000-0000-0000-000000000000,"((Text(""ПМЭФ 2023"")) AND ((Metadata(SourceLeve...",[ПМЭФ 2023],[]
8,00000000-0000-0000-0000-000000000000,"((((Org(Id(8CBC8)))) AND NOT(Text(""вагенровцы""...","[вагенровцы, вск-милосердие]","[вагенровцы, вск-милосердие]"
9,00000000-0000-0000-0000-000000000000,(((((Metadata(Id(825CFC))) OR (Metadata(Id(825...,"[прогноз ввп, бюджет РФ, бюджет россии, Мой ра...",[Мой район]


In [60]:
for el in list(df[8:9].QueryExcludeText):
    print(el)
#     list(df[9:10].QueryText).remove (el)

['вагенровцы', 'вск-милосердие']


In [64]:
for el in list(['вагенровцы', 'вск-милосердие']):
    print(el)

вагенровцы
вск-милосердие


In [13]:
# удаляем скобки и заменяем пробелами
df['Query'] = df['Query'].apply(lambda query: query.replace('(', ' '))
df['Query'] = df['Query'].apply(lambda query: query.replace(')', ' '))


In [14]:
# выделение текста запроса через регулярку. Возможно, ее можно доработать.
# не организации, не персоны, не локации = ['Org', 'Name', 'Location']

def get_txt(query):
  query.split(' ')
  pattern = r'["]\S+\s*\S+\s*\S+["]'       #до 3 слов
  result = re.findall(pattern, query)
  return result


df['Text'] = df['Query'].apply(lambda query: get_txt(query)).astype(str)

In [15]:
df['Text'] = df['Text'].apply(lambda x: x.split(','))             # делим строку по запятой
df = df.explode('Text', ignore_index=True)                        # переносим текст на новую строку
df['Text'] = df['Text'].astype(str)

In [16]:
def edit_text(text):
  '''Функция, удаляющая знаки, которые не входят в состав фразы '''
  del_symbols = ['[', ']', '"', "'", '”', '“', '#', '*', '•t', '•txxx']
  for symbol in del_symbols:
    text = text.replace(symbol, '')
  return text

df['Text'] = df['Text'].apply(lambda text: edit_text(text))

In [17]:
 # удаляем строки с пустыми запросами
 del_index = list(df[df.Text == ''].index)
 df = df.drop(index=del_index)

In [63]:
df.sample(5)

,UserId,Query,QueryText,QueryExcludeText
25712,4608aee8-1271-eb11-8ff1-cf98f80a40f6,"((Text(""Сегежа Сибирь"")) OR (Text(""СегежаСибир...","[Сегежа Сибирь, СегежаСибирь, Сегежа-Сибирь]",[]
11010,8dc1a86f-92b5-e611-9e84-00155d015200,"((((((Org(Id(69D508))) OR (Text(""ВТБ Беларусь""...","[ВТБ Беларусь, VTB Belarus, Беларусская дочка ...",[Единой лиге ВТБ]
14766,bade7480-9d0e-ee11-8275-872c80af2753,"((((InSameParagraph(""регулирование законодател...","[44-фз, 223-фз, госзакупки, государственные за...",[]
23164,3280d24b-cb85-ec11-a277-f63e450595c3,"((Text(""Вице-президент ЕК считает, что Украина...","[Вице-президент ЕК считает, что Украина сможет...",[]
16820,a76180dd-e7ea-e711-a815-a31faa6cf031,"(((Org(Id(47A2B3))) AND NOT(((InTitle(Text(""ры...","[рынок акций, лидеры роста/падения, итоги торгов]",[]


Выделить группы похожих запросов у разных пользователей (по UserId)

In [39]:
# текст запроса и количество уникальных юзеров, которые набрали его
df.groupby(['Text'])['UserId'].nunique().sort_values(ascending=False).head(50)

Text
9ff8d3d3-7624-490e-acd8-bfb2e3cf1edf     33
 закон                                   27
 9ff8d3d3-7624-490e-acd8-bfb2e3cf1edf    26
 футбол                                  25
 акции                                   23
 законопроект                            23
 газ                                     19
 рынок акций                             18
 инвестиции                              18
 банк                                    18
 суд                                     17
 санкции                                 17
 школа                                   17
 завод                                   17
 спецоперация                            16
 авария                                  16
 жкх                                     16
 нефть                                   16
 строительство                           15
 Украина                                 15
 военная операция                        15
 путин                                   15
 самолет                   

Для выделенных групп запросов по "облакам слов" выработать метрику, максимизирующую качество маппинга некоторых из выделенных облаков на предоставленные тематики.

In [50]:
# проверим первые 1000 популярных запросов
count = 1000

# сoставим список популярных слов в запросе
popular_queries =pd.DataFrame(
    data=list(df.groupby(['Text'])['UserId'].nunique().sort_values(ascending=False).index[0:count-1]),
    columns=['Text']
)
popular_queries = popular_queries.Text.apply(lambda text: text.split())         # разобьем по словам
popular_queries = popular_queries.explode()                                     # перенесем каждое новое слово
popular_queries = list(popular_queries)

# создадим счетчики найденных сопадений
in_clouds = Counter()                  #если слово есть в облаках слов
not_found = Counter()                  #если слово не найдено в облаках слов

# переводим слова в нижний регистр и ищем совпадения
for i in range(0, len(popular_queries)):
  for j in range(0, 572):
    if popular_queries[i].lower() in words_clouds_data[j]['WordCloud']:
      in_clouds[popular_queries[i]] += 1
    else:
      not_found[popular_queries[i].lower()] += 1

print(f'Найдено совпадений {len(in_clouds)} из {len(in_clouds)+len(not_found)}.')

Найдено совпадений 488 из 1475.


In [54]:
# самые распространенные ненайденные значения
not_found.most_common()

[('text', 8580),
 ('строительство', 6903),
 ('росси', 6292),
 ('аэс', 5700),
 ('рф', 4004),
 ('цеха', 4004),
 ('investments', 4004),
 ('рейтинг', 3948),
 ('limited', 3432),
 ('бизнес', 3330),
 ('комплекс', 3222),
 ('втб', 2860),
 ('цифровая', 2860),
 ('ltd', 2860),
 ('информационная', 2855),
 ('акций', 2835),
 ('проект', 2750),
 ('магазин', 2720),
 ('центр', 2695),
 ('пмэф', 2288),
 ('газпром', 2288),
 ('санкции', 2284),
 ('железнодорожные', 2284),
 ('премьера', 2280),
 ('россии', 2276),
 ('технологии', 2276),
 ('перевозки', 2272),
 ('безопасность', 2268),
 ('ремонт', 2236),
 ('поддержка', 2232),
 ('рынок', 2084),
 ('банк', 2076),
 ('украина', 1716),
 ('сво', 1716),
 ('лизинг', 1716),
 ('обзор', 1716),
 ('котировки', 1716),
 ('россети', 1716),
 ('северный', 1716),
 ('wildcard', 1716),
 ('цифровые', 1716),
 ('добрынинский', 1716),
 ('белград', 1716),
 ('молодёжный', 1716),
 ('долгопрудный', 1716),
 ('инвестментс', 1716),
 ('приостанавливает', 1716),
 ('ростелеком', 1713),
 ('порт', 1710